# Day 2: Full Fine-Tuning

This notebook implements **Full Fine-Tuning** where we update ALL parameters of the model.

**Model**: meta-llama/Llama-2-7b-hf or mistralai/Mistral-7B-v0.1

**Expected Time**: 3-4 hours

**GPU Required**: T4 (15GB) or A100

**Target Accuracy**: 85-95%

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q torch transformers accelerate datasets evaluate scikit-learn pandas numpy wandb trl

In [ ]:
# Import libraries
import torch
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, classification_report
import wandb

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Mount Google Drive and Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load data
data_path = '/content/drive/MyDrive/Colab Notebooks/llm-finetuning-showdown/processed'

train_df = pd.read_csv(f'{data_path}/train.csv')
val_df = pd.read_csv(f'{data_path}/val.csv')
test_df = pd.read_csv(f'{data_path}/test.csv')

with open(f'{data_path}/label_mapping.json', 'r') as f:
    label_info = json.load(f)

# Convert text labels to numeric IDs
label_to_id = label_info['label_to_id']
train_df['label'] = train_df['label'].map(label_to_id)
val_df['label'] = val_df['label'].map(label_to_id)
test_df['label'] = test_df['label'].map(label_to_id)

print(f"✅ Train samples: {len(train_df)}")
print(f"✅ Val samples: {len(val_df)}")
print(f"✅ Test samples: {len(test_df)}")
print(f"\n✅ Number of categories: {label_info['num_labels']}")
print(f"✅ Categories: {list(label_info['label_to_id'].keys())}")
print(f"\n✅ Labels converted to numeric IDs")
print(f"   First label (should be 0-24): {train_df['label'].iloc[0]}")

## 3. Initialize Weights & Biases

In [ ]:
# Login to W&B
wandb.login()

# Initialize project
wandb.init(
    project="llm-finetuning-showdown",
    name="full-finetuning",
    config={
        "method": "full_finetuning",
        "model": "mistralai/Mistral-7B-v0.1",
        "task": "resume_classification",
        "num_labels": label_info['num_labels'],
        "learning_rate": 2e-5,
        "batch_size": 4,
        "epochs": 3,
        "max_length": 512
    }
)

## 4. Load Model and Tokenizer

In [ ]:
# Choose model (Mistral is faster and better than Llama-2)
model_name = "mistralai/Mistral-7B-v0.1"
# Alternative: "meta-llama/Llama-2-7b-hf" (requires HuggingFace access token)

print(f"Loading model: {model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model for classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=label_info['num_labels'],
    torch_dtype=torch.bfloat16,  # Use BF16 from the start
    device_map="auto"
)

# Configure pad token id
model.config.pad_token_id = tokenizer.pad_token_id

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

print(f"✅ Model loaded: {model_name}")
print(f"✅ Total parameters: {model.num_parameters():,}")
print(f"✅ Trainable parameters: {model.num_parameters():,}")
print(f"✅ Gradient checkpointing: Enabled")

## 5. Prepare Dataset

In [ ]:
# Tokenization function - NO PADDING HERE (let Trainer handle it)
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=512
    )

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
print("Tokenizing datasets...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Clean up: Remove text column (no longer needed)
train_dataset = train_dataset.remove_columns(['text'])
val_dataset = val_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])

# Rename 'label' to 'labels' (required by Trainer)
train_dataset = train_dataset.rename_column('label', 'labels')
val_dataset = val_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(f"✅ Datasets tokenized and ready")
print(f"   Final columns: {train_dataset.column_names}")

## 6. Define Metrics and Training Arguments

In [ ]:
# Compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted'
    )
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments - Memory optimized for A100 80GB with gradient checkpointing
training_args = TrainingArguments(
    output_dir="./results_full_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduced from 8 to 4 for memory
    per_device_eval_batch_size=8,   # Reduced from 16 to 8 for memory
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    warmup_steps=100,
    fp16=False,  # Disabled for A100
    bf16=True,   # Use BF16 - native A100 support
    gradient_checkpointing=True,  # Save memory by recomputing activations
    gradient_accumulation_steps=2,  # Maintain effective batch size of 8
    report_to="wandb",
    run_name="full-finetuning"
)

print("✅ Training arguments configured")
print("🚀 Memory optimized for A100 80GB:")
print("   - Batch size: 4 (effective: 8 with accumulation)")
print("   - Gradient checkpointing: Enabled")
print("   - BF16 precision")

## 7. Train Model

In [ ]:
# Create data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("🚀 Starting training...")
print(f"📊 Training samples: {len(train_dataset)}")
print(f"📊 Validation samples: {len(val_dataset)}")
print(f"⏰ Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Track training time
start_time = time.time()

# Train
train_result = trainer.train()

# Calculate training time
training_time = time.time() - start_time
training_hours = training_time / 3600

print(f"\n✅ Training completed!")
print(f"⏰ Training time: {training_hours:.2f} hours ({training_time:.2f} seconds)")
print(f"📈 Final training loss: {train_result.training_loss:.4f}")

## 8. Evaluate on Test Set

In [ ]:
# Evaluate on test set
print("🧪 Evaluating on test set...")
test_results = trainer.evaluate(test_dataset)

print("\n" + "="*50)
print("FULL FINE-TUNING RESULTS")
print("="*50)
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f} ({test_results['eval_accuracy']*100:.2f}%)")
print(f"Test F1-Score: {test_results['eval_f1']:.4f}")
print(f"Test Precision: {test_results['eval_precision']:.4f}")
print(f"Test Recall: {test_results['eval_recall']:.4f}")
print(f"\nTraining Time: {training_hours:.2f} hours")
print(f"Baseline Accuracy: 73.00%")
print(f"Improvement: +{(test_results['eval_accuracy']*100 - 73):.2f}%")

# Get detailed predictions
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Classification report
print("\nDetailed Classification Report:")
id_to_label = {v: k for k, v in label_info['label_to_id'].items()}
target_names = [id_to_label[i] for i in range(label_info['num_labels'])]
print(classification_report(true_labels, pred_labels, target_names=target_names))

## 9. Save Results and Model

In [ ]:
# Save results to Google Drive
results_path = '/content/drive/MyDrive/Colab Notebooks/llm-finetuning-showdown'

full_ft_results = {
    "method": "full_finetuning",
    "model": model_name,
    "date": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "accuracy": float(test_results['eval_accuracy']),
    "f1_score": float(test_results['eval_f1']),
    "precision": float(test_results['eval_precision']),
    "recall": float(test_results['eval_recall']),
    "training_time_hours": float(training_hours),
    "training_time_seconds": float(training_time),
    "baseline_accuracy": 0.73,
    "improvement_over_baseline": float(test_results['eval_accuracy'] - 0.73),
    "total_parameters": model.num_parameters(),
    "trainable_parameters": model.num_parameters(),
    "config": {
        "learning_rate": 2e-5,
        "batch_size": 4,
        "epochs": 3,
        "max_length": 512
    }
}

with open(f'{results_path}/full_ft_results.json', 'w') as f:
    json.dump(full_ft_results, f, indent=2)

print(f"✅ Results saved to: {results_path}/full_ft_results.json")

# Save model (optional - large file)
# model.save_pretrained(f'{results_path}/full_ft_model')
# tokenizer.save_pretrained(f'{results_path}/full_ft_model')
# print(f"✅ Model saved to: {results_path}/full_ft_model")

# Log to W&B
wandb.log({
    "final_test_accuracy": test_results['eval_accuracy'],
    "final_test_f1": test_results['eval_f1'],
    "training_time_hours": training_hours,
    "improvement_over_baseline": test_results['eval_accuracy'] - 0.73
})

wandb.finish()
print("\n✅ Full Fine-Tuning Complete!")

## 10. Next Steps

**✅ Full Fine-Tuning Complete!**

**Record your results:**
- Accuracy: ____%
- Training time: ___ hours
- GPU memory used: ___ GB (check nvidia-smi)

**Next:**
- Run LoRA fine-tuning (Day2_LoRA_FineTuning.ipynb)
- Compare efficiency and accuracy